## Compare the results between paramlink2 and mlink

## the results of paramlink2 and the results of mlink are consistent

In [ ]:
import sys
import os.path
import glob
import pandas as pd
import numpy as np
import pickle
from SEQLinkage.linkage import *

In [ ]:
from SEQLinkage.Utils import *

In [ ]:
from SEQLinkage.Runner import *

In [ ]:
fam_vcf='../data/wg20220316/fam17_vcf.pickle'
fam_path='../data/new_trim_ped_famless17_no:xx.fam'
if os.path.isfile(fam_vcf):
    with open(fam_vcf, 'rb') as handle:
        fam17_vcf = pickle.load(handle)
fam17 = pd.read_csv(fam_path,delim_whitespace=True,header=None,names=['fid','iid','fathid','mothid','sex','ad'])
fam17.index = list(fam17.iid)
fam17.ad[fam17.ad==-9]=0
fam17_d = {}
cutoff=0.05
for i in fam17.fid.unique():
    fam17_d[i] = fam17[fam17.fid==i]

/tmp/2485903.1.high_mem.q/ipykernel_20176/2986720832.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fam17.ad[fam17.ad==-9]=0


### generate mlink input

In [ ]:
#linkage format, .pre and .loc
#per locus, per family based
def format_linkage(out_base, haps, variants, prev=0.001, wild_pen=0.01, muta_pen=0.9, inherit_mode='AD', theta_max=0.5, theta_inc=0.05):
        #disease model
        heter_pen = wild_pen
        if inherit_mode == 'AD':
            heter_pen = muta_pen
            
        for fid in variants.keys():
            for i,v in variants[fid].iterrows():
                if not v.uniq:
                    continue
                workdir = '{}/{}/{}'.format(out_base, v.variants, fid)
                mkpath(workdir)
                
                gs_num=2
                fam_af = [str(1-round(v.freqs,3)),str(round(v.freqs,3))]
                
                #write to .PRE
                pre=haps[fid].loc[:,['fid','iid','fathid','mothid','sex','ad',v.variants+'_A0',v.variants+'_A1']]
                pre.to_csv('{}/{}.PRE'.format(workdir, '1'),header=False,index=False,sep=' ')
               
                #write to .LOC
                with open('{}/{}.LOC'.format(workdir, '1'), 'w') as loc:
                    loc.write("2 0 0 5\n")
                    loc.write("0 0.0 0.0 0\n")
                    loc.write("1 2\n")
                    loc.write("1 2\n")
                    loc.write(" {} {}\n".format(1 - prev, prev))
                    loc.write(" 1\n")
                    loc.write(" {} {} {}\n".format(wild_pen, heter_pen, muta_pen))
                    loc.write("3 {}\n".format(gs_num))  #the number of allele types, int
                    loc.write(' ' + ' '.join(fam_af) + "\n") #[a0_freq,a1_freq]
                    loc.write("0 0\n")
                    loc.write("0.0\n")
                    loc.write("1 {} {}\n".format(theta_inc, theta_max))

In [ ]:
with open('../data/wg20220425genes/chr22test/tmp/CACHE/chr22test25cutoff0.05unimputed.input', 'rb') as handle:
    gene_variants,gene_fam_haps = pickle.load(handle) 

In [ ]:
format_linkage('../data/wg20220425genes/LINKAGE',gene_fam_haps,gene_variants)

### calculate lods by mlink

In [ ]:
env.outdir='../data/wg20220425genes/'

In [ ]:
workdir='../data/wg20220425genes/LINKAGE'

In [ ]:
env.jobs=1

In [ ]:
linkage_worker('../data/wg20220425genes/tenvariants_blutprint.txt',workdir,0.05,0.5,'../data/wg20220425genes/LinkageRuntimeError.txt',to_plot=False)

### compare the results

In [ ]:
with open('../data/wg20220425genes/chr22test/tmp/CACHE/chr22test25cutoff0.05unimputed.lods', 'rb') as handle:
    lods = pickle.load(handle) 

In [ ]:
lods['723'].loc['chr22:50697518:G:GGCCC_A0',:]

LOD0.0     0.016533
LOD0.05    0.013439
LOD0.1     0.010653
LOD0.15    0.008180
LOD0.2     0.006024
LOD0.25    0.004192
LOD0.3     0.002688
LOD0.35    0.001514
LOD0.4     0.000674
LOD0.45    0.000168
Name: chr22:50697518:G:GGCCC_A0, dtype: float64

In [ ]:
mlinklods=pd.read_csv('../data/wg20220425genes/heatmap/LINKAGE.family-lods',header=None,sep=' ')

In [ ]:
mlinklods.columns=['fid','variants','chr','start','end','theta','mlods']

In [ ]:
'LOD'+str(r.theta)

'LOD0.05'

In [ ]:
plods=[]
for i,r in mlinklods.iterrows():
    if r.theta==0.5:
        plods.append(0.0)
    else:
        plods.append(lods[r.fid].loc[r.variants+'_A0','LOD'+str(r.theta)])

In [ ]:
mlinklods['plods']=plods

In [ ]:
sum(np.abs(mlinklods.mlods-mlinklods.plods)>0.00001)

37